# <center> Design Notebook

## General Approach

Use a cv2 while ret is True loop to process entire video
- call object detection function (creates bounding boxes)
- call counter function (check if bounding box intersects ROI)
- call drawing functions (ROI line, bounding boxes, counter text)
- return output video

### Object Detection Function

- Import SSD_MobileNet_V1
- Create label dictionary for converting detection classes to label strings
- Output vertices for every detected vehicle or person *(for Drawing Functions)*
- Compute and store average/center of two vertices *(for Counter Function)*
    - use top-left and top-right vertices for vertical ROI
    - use top-right and bottom-right vertices for horizontal ROI

### Counter Function

Setting a ROI range that is dynamic based on width of the video should be able to account for faster
vehicles and flickering bounding boxes. Center of bounding boxes will be used since the center of
bounding boxes should vary less than vertices. Videos should all be 30 FPS but range may need to be
adjusted depending on speed of vehicles or pedestrians. Since pedestrians are slower than vehicles,
pedestrians may require a smaller ROI to avoid double counting.

if box_center is within a certain range of the width +/- 2% of width then increase counter by 1

Will also return is_vehicle_detected as boolean *(for Drawing Functions)*

```
if box_center in range(int(width/2 - .02 * width),
                       int(width/2 + .02 * width)):
    counter += 1
    is_vehicle_detected = True
else:
    is_vehicle_detected = False
```

### Drawing Functions

#### ROI Line

Will have a conditional for a vertical line or horizontal line.
If is_vehicle_detected is True then ROI line will be green, otherwise it will be red.


```
if ROI_Line == 'vertical':
    roi_start = (int(width/2), 0)
    rot_end = (int(width/2), height)
else:
    roi_start = (0, int(height/2))
    rot_end = (width, int(height/2))

if is_vehicle_detected is True:
    cv2.line(input_frame, roi_start, roi_end, (0, 0, 0xFF), 5)
else:
    cv2.line(input_frame, roi_start, roi_end, (0, 0xFF, 0), 5)
```

#### Bounding Boxes

Investigate implementation in reference API and decide whether to use
Pillow (reference API) or cv2.rectangle (new implementation from scratch)

```
# Actual detection
(boxes, scores, classes, num) = \
    sess.run([detection_boxes, detection_scores,
             detection_classes, num_detections],
             feed_dict={image_tensor: image_np_expanded})

# Visualization of the results of a detection.
(counter, csv_line) = \
    vis_util.visualize_boxes_and_labels_on_image_array(
    cap.get(1),
    input_frame,
    np.squeeze(boxes),
    np.squeeze(classes).astype(np.int32),
    np.squeeze(scores),
    category_index,
    roi_info,
    use_normalized_coordinates=True,
    line_thickness=4,
    )
```


#### Counter Text

Could be fine-tuned to be more dynamic and not be set to precise pixels

```
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(
    input_frame,
    'Detected Vehicles: ' + str(counter),
    (10, 35),
    font,
    0.8,
    (0, 0xFF, 0xFF),
    2,
    cv2.FONT_HERSHEY_SIMPLEX,
    )
 ```


#### Bounding Boxes

Investigate implementation in reference API and decide whether to use
Pillow (reference API) or cv2.rectangle (new implementation from scratch)

```
# Actual detection
(boxes, scores, classes, num) = \
    sess.run([detection_boxes, detection_scores,
             detection_classes, num_detections],
             feed_dict={image_tensor: image_np_expanded})

# Visualization of the results of a detection.
(counter, csv_line) = \
    vis_util.visualize_boxes_and_labels_on_image_array(
    cap.get(1),
    input_frame,
    np.squeeze(boxes),
    np.squeeze(classes).astype(np.int32),
    np.squeeze(scores),
    category_index,
    roi_info,
    use_normalized_coordinates=True,
    line_thickness=4,
    )
```

https://stackoverflow.com/questions/48915003/get-the-bounding-box-coordinates-in-the-tensorflow-object-detection-api-tutorial

```
Adds a bounding box to an image.
  Bounding box coordinates can be specified in either absolute (pixel) or
  normalized coordinates by setting the use_normalized_coordinates argument.
  Each string in display_str_list is displayed on a separate line above the
  bounding box in black text on a rectangle filled with the input 'color'.
  If the top of the bounding box extends to the edge of the image, the strings
  are displayed below the bounding box.
  Args:
    image: a PIL.Image object.
    ymin: ymin of bounding box.
    xmin: xmin of bounding box.
    ymax: ymax of bounding box.
    xmax: xmax of bounding box.
    color: color to draw bounding box. Default is red.
    thickness: line thickness. Default value is 4.
    display_str_list: list of strings to display in box
                      (each to be shown on its own line).
    use_normalized_coordinates: If True (default), treat coordinates
      ymin, xmin, ymax, xmax as relative to the image.  Otherwise treat
      coordinates as absolute.
  ```

**The following call sequence draws the bounding boxes**

def draw_bounding_box_on_image_array

 ```
  # Draw all boxes onto image.
  for box, color in box_to_color_map.items():
    ymin, xmin, ymax, xmax = box
    if instance_masks is not None:
      draw_mask_on_image_array(
          image,
          box_to_instance_masks_map[box],
          color=color
      )
```

  ```
  image_pil = Image.fromarray(np.uint8(image)).convert('RGB')
  is_vehicle_detected, csv_line, update_csv = draw_bounding_box_on_image(current_frame_number,image_pil, ymin, xmin, ymax, xmax, roi_info, color,
                             thickness, display_str_list,
                             use_normalized_coordinates)
  np.copyto(image, np.array(image_pil))
  return is_vehicle_detected, csv_line, update_csv
```

def draw_bounding_box_on_image

```
  image_temp = numpy.array(image)
  csv_line = "" # to create new csv line consists of vehicle type, predicted_speed, color and predicted_direction
  update_csv = False # update csv for a new vehicle that are passed from ROI - just one new line for each vehicles
  is_vehicle_detected = [0]
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  if use_normalized_coordinates:
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
  else:
    (left, right, top, bottom) = (xmin, xmax, ymin, ymax)
  draw.line([(left, top), (left, bottom), (right, bottom),
             (right, top), (left, top)], width=thickness, fill=color)

  detected_vehicle_image = image_temp[int(top):int(bottom), int(left):int(right)]
  ```

Boxes should be in the form of

<center> [xmin,xmax,ymin,ymax]<center>

To calculate `box_center`, use coordinates to calculate the center